## Scratch

In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs
import tf_lab.point_clouds.in_out as pio
import tf_lab.models.point_net_based_AE as pnAE
from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id
from tf_lab.point_clouds.in_out import PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.autopredictors.evaluate.basics import eval_model, read_saved_epochs
from tf_lab.autopredictors.plotting.basics import plot_original_pclouds_vs_reconstructed,\
                                                  plot_train_val_test_curves, plot_reconstructions_at_epoch

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from geo_tool import Point_Cloud

seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
DATA_PATH = '/orions4-zfs/projects/lins2/Panos_Space/DATA/ShapeNetPointClouds/from_manifold_meshes/1024/03001627/'
TRAIN_DIR = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/vanilla_ae/chairs_only/'

experiment_name = 'enc_filter_1_complex_decoder'
TRAIN_DIR = osp.join(TRAIN_DIR, experiment_name)
# create_dir(TRAIN_DIR)

file_names = pio.load_filenames_of_input_data(DATA_PATH)
all_pclouds, model_names, _ = pio.load_crude_point_clouds(file_names=file_names, n_threads=10)


conf = Conf(n_input = [1024, 3],
            training_epochs = 1000,
               batch_size = 40,
               loss = 'Chamfer',
               train_dir = TRAIN_DIR,
               loss_display_step = 1,
               saver_step = 5,
               learning_rate = 0.00002,
               saver_max_to_keep = 200,
               gauss_augment = {'mu': 0, 'sigma': 0.02},
               encoder = pnAE.encoder,
               decoder = pnAE.decoder
               )


train_data_, val_data_, test_data_ = pio.train_validate_test_split([all_pclouds,                                                             
                                                            model_names],
                                                            train_perc=0.8,
                                                            validate_perc=0.1,
                                                            test_perc=0.1,
                                                            seed=seed)

train_data = PointCloudDataSet(train_data_[0], labels=train_data_[1])
val_data = PointCloudDataSet(val_data_[0], labels=val_data_[1])
test_data = PointCloudDataSet(test_data_[0], labels=test_data_[1])

In [3]:
ae = PointNetAutoEncoder(experiment_name, conf)

In [5]:
saved_epochs = read_saved_epochs(conf.train_dir)
epochs = saved_epochs[::len(saved_epochs)/20]
stats = eval_model(ae, conf, [train_data, test_data], epochs=epochs, verbose=True)

[ 0.0037341   0.00369484]
[ 0.00201771  0.00205854]
[ 0.00168847  0.00171357]
[ 0.00155746  0.00158419]
[ 0.00148309  0.00151353]
[ 0.00142828  0.00145644]
[ 0.00139644  0.00142714]
[ 0.00136612  0.00139871]
[ 0.00134623  0.00137464]
[ 0.00132668  0.00135893]
[ 0.00131309  0.00133641]
[ 0.00129739  0.00133096]
[ 0.00128579  0.00131396]
[ 0.00127496  0.00130205]
[ 0.00126463  0.00130055]
[ 0.00125738  0.00129662]
[ 0.00124866  0.00128835]
[ 0.00124188  0.00127236]
[ 0.0012388   0.00127656]
[ 0.00123025  0.00127712]


In [12]:
ae.restore_model(conf.train_dir, 995)
ae.evaluate(train_data, conf)

AttributeError: Configuration instance has no attribute '__str__'

In [ ]:



### Evaluation

SAVEDIR = osp.join('/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/model_evaluation/vanilla_ae', experiment_name)
create_dir(SAVEDIR)

gen_error, best_epoch, stats = generalization_error(ae, train_data, test_data, val_data, conf)
print 'Best generalization error %f (at epoch %d)' % (gen_error, best_epoch)
np.savetxt(osp.join(SAVEDIR, 'error-stats.txt'), stats)
plot_train_val_test_curves(stats, SAVEDIR, best_epoch, show=False)

plot_reconstructions_at_epoch(best_epoch, ae, test_data, conf, save_dir=osp.join(SAVEDIR, 'test_images_epoch_' + str(best_epoch)))
plot_reconstructions_at_epoch(best_epoch, ae, train_data, conf, save_dir=osp.join(SAVEDIR, 'train_images_epoch_' + str(best_epoch)), max_plot=100)

last_epoch = stats[-1, 0]
plot_reconstructions_at_epoch(last_epoch, ae, test_data, conf, save_dir=osp.join(SAVEDIR, 'test_images_epoch' + str(last_epoch)))  # Also print models at last epoch.



In [37]:
# Experimenting with Tensorboard - Visualizations
from tf_lab.fundamentals.inspect import hist_summary_of_trainable
writer = tf.summary.FileWriter('/Users/optas/Desktop/test', ae.graph)
# hist_summary_of_trainable(ae.graph)